In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

#save path
dir_path = '/Users/sinksarghebremedhin/Desktop/Tensorflow-Guide SIJACO'

#load dataset
mnist = input_data.read_data_sets(dir_path + '/data/', one_hot=True)
train_size = mnist.train.num_examples
test_size = mnist.test.num_examples

Extracting /Users/sinksarghebremedhin/Desktop/Tensorflow-Guide SIJACO/data/train-images-idx3-ubyte.gz
Extracting /Users/sinksarghebremedhin/Desktop/Tensorflow-Guide SIJACO/data/train-labels-idx1-ubyte.gz
Extracting /Users/sinksarghebremedhin/Desktop/Tensorflow-Guide SIJACO/data/t10k-images-idx3-ubyte.gz
Extracting /Users/sinksarghebremedhin/Desktop/Tensorflow-Guide SIJACO/data/t10k-labels-idx1-ubyte.gz


In [2]:
def display_digit(image, label=None, pred_label=None):
    if image.shape ==(784,):
        image = image.reshape((28,28))
    label = np.argmax(label, axis=0)
    if pred_label is None and label is not None:
        plt.title('Label: %d' %(label))
    elif label is not None:
        plt.title('Label: %d, Pred: %d' %(label, pred_label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()

def display_convergence(error):
    plt.plot(error)
    plt.title('Error of the NN')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.show()

In [3]:
#variablen
epochs = 30
#batch size in cuda am besten 2er potenz 
batch_size = 256
classes = 10
features = 784
#wieviele neuronen in verschiedenen Layern
layer_nodes = [features, 200, 400, 200, classes]
#standardabweichung
stddev = 0.010
bias_weight_init = 0.50
learning_rate = 1e-4


In [4]:
# TF Placeholders # warum none in shape ??
x = tf.placeholder(dtype=tf.float32, shape=[None, features], name='x')
y = tf.placeholder(dtype=tf.float32, shape=[None, classes], name='y')

In [5]:
# Weight Matrices
W1 = tf.Variable(tf.truncated_normal([layer_nodes[0],
                                    layer_nodes[1]],
                                    stddev=stddev, name="W1"))

W2 = tf.Variable(tf.truncated_normal([layer_nodes[1],
                                    layer_nodes[2]],
                                    stddev=stddev, name="W2"))

W3 = tf.Variable(tf.truncated_normal([layer_nodes[2],
                                    layer_nodes[3]],
                                    stddev=stddev, name="W3"))

W4 = tf.Variable(tf.truncated_normal([layer_nodes[3],
                                    layer_nodes[4]],
                                    stddev=stddev, name="W4"))

In [6]:
# Biases Vectors
b1 = tf.Variable(tf.truncated_normal([layer_nodes[1]],
                                    stddev=stddev, name="b1"))

b2 = tf.Variable(tf.truncated_normal([layer_nodes[2]],
                                    stddev=stddev, name="b2"))

b3 = tf.Variable(tf.truncated_normal([layer_nodes[3]],
                                    stddev=stddev, name="b3"))

b4 = tf.Variable(tf.truncated_normal([layer_nodes[4]],
                                    stddev=stddev, name="b4"))

In [7]:

# Define the Deep Neural Network
def nn_model(x):
    input_layer      = {'weights': W1, 'biases': b1}
    hidden_layer_1   = {'weights': W2, 'biases': b2}
    hidden_layer_2   = {'weights': W3, 'biases': b3}
    output_layer     = {'weights': W4, 'biases': b4}
   

    input_layer_sum = tf.add(tf.matmul(x, input_layer['weights']),
                            input_layer['biases'])
    input_layer_sum = tf.nn.relu(input_layer_sum)

    #hiddenlayer1
    hidden_layer_1_sum = tf.add(tf.matmul(input_layer_sum, hidden_layer_1['weights']),
                            hidden_layer_1['biases'])
    hidden_layer_1_sum = tf.nn.relu(hidden_layer_1_sum)

    #hiddenlayer2
    hidden_layer_2_sum = tf.add(tf.matmul(hidden_layer_1_sum, hidden_layer_2['weights']),
                            hidden_layer_2['biases'])
    hidden_layer_2_sum = tf.nn.relu(hidden_layer_2_sum)

    #output layer
    output_layer_sum = tf.add(tf.matmul(hidden_layer_2_sum, output_layer['weights']),
                            output_layer['biases'])
    
    return output_layer_sum

In [8]:
#train the network
def nn_train(x):
    epoch_errors = []
    pred = nn_model(x)
    pred = tf.identity(pred)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #Training
        for epoch in range(epochs):
            epoch_loss = 0.0
            
            for i in range(int(train_size / batch_size)+1):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            epoch_errors.append(epoch_loss)
            print("epoch: " , epoch + 1, " of " , epochs , " with loss: " , epoch_loss)
        #display_convergence(epoch_errors)
        #Testing
        correct_result = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_result,'float'))
        print('Accuracy: ', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))


In [9]:
nn_train(x)

epoch:  1  of  30  with loss:  334.270492613
epoch:  2  of  30  with loss:  145.380720228
epoch:  3  of  30  with loss:  125.319060504
epoch:  4  of  30  with loss:  113.378818244
epoch:  5  of  30  with loss:  103.196672022
epoch:  6  of  30  with loss:  93.5043314397
epoch:  7  of  30  with loss:  82.8261051774
epoch:  8  of  30  with loss:  72.6026547998
epoch:  9  of  30  with loss:  64.967619583
epoch:  10  of  30  with loss:  59.3134810627
epoch:  11  of  30  with loss:  54.7872754782
epoch:  12  of  30  with loss:  51.0232528895
epoch:  13  of  30  with loss:  47.5441974849
epoch:  14  of  30  with loss:  44.7148006782
epoch:  15  of  30  with loss:  41.7006443292
epoch:  16  of  30  with loss:  39.1209239885
epoch:  17  of  30  with loss:  37.3184373453
epoch:  18  of  30  with loss:  34.979401961
epoch:  19  of  30  with loss:  33.2270972356
epoch:  20  of  30  with loss:  31.4400191009
epoch:  21  of  30  with loss:  30.1079876497
epoch:  22  of  30  with loss:  28.4582511447